In [1]:
import importlib

import training
import utils
import training_constants as tc
import preprocessing as pre
# Avaliar e selecionar o melhor seletor de variáveis
from sklearn.pipeline import Pipeline
import feature_selection as fs
import evaluation

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
importlib.reload(training)
importlib.reload(utils)
importlib.reload(tc)
importlib.reload(pre)
importlib.reload(fs)
importlib.reload(training)
importlib.reload(evaluation)


<module 'evaluation' from '/Users/patricia/Documents/code/python/behavior-detection/src/evaluation.py'>

In [3]:
import os
import warnings
import numpy as np

# Configurar variáveis de ambiente para não mostrar warning da Intel MKL
os.environ["MKL_CBWR"] = "AUTO"
os.environ["MKL_VERBOSE"] = "0"
os.environ["NUMPY_MKL_ERROR"] = "IGNORE"
os.environ["MKL_SERVICE_FORCE_INTEL"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["MKL_DYNAMIC"] = "FALSE"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["KMP_WARNINGS"] = "FALSE"

# Suprimir avisos de depreciação
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
X, y = pre.load_data("/Users/patricia/Documents/code/python/behavior-detection/data/new_logs_labels.csv")
y.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5489 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.1+ MB
<class 'pandas.core.series.Series'>
Index: 5489 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5489 non-null   object
dtypes: object(1)
memory usage: 85.8+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [5]:
test_size = 0.2  # 80% for training, 20% for testing
X_train, X_test, y_train, y_test = pre.split_train_test_data(X, y, test_size, random_state=42)


Tamanho do conjunto de treino: 4391
Tamanho do conjunto de teste: 1098


In [6]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train).value_counts())
print("\n\nNro de instancias de cada classe em y_test:\n")
print(pd.Series(y_test).value_counts())

Nro de instancias de cada classe em y_train:

ON TASK                 2553
ON SYSTEM                721
OFF TASK                 495
ON TASK CONVERSATION     321
ON TASK OUT              301
Name: count, dtype: int64


Nro de instancias de cada classe em y_test:

ON TASK                 606
ON SYSTEM               186
OFF TASK                134
ON TASK CONVERSATION     93
ON TASK OUT              79
Name: count, dtype: int64


In [7]:
y_train, label_encoder = pre.encode_labels(y_train)
y_test = label_encoder.transform(y_test)

In [8]:
# Pré-processar os dados uma vez
preprocessor = pre.create_preprocessor(X_train)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [9]:
best_selector, best_selector_name = fs.evaluate_feature_selectors(X_train_preprocessed, y_train, n_features_to_select=10, n_components=10)

# Exibir o melhor seletor e as características selecionadas
print(f'O melhor seletor de variáveis é: {best_selector_name}')



O melhor seletor de variáveis é: RandomForest


In [10]:
# Adicionar o melhor seletor de variáveis no pipeline de treinamento
base_pipeline = Pipeline([
    ('feature_selection', best_selector)
])

In [11]:
# Bayesian optimization não está funcionando corretamente. Usar Random Search ou Grid Search
trained_models = training.train_model(X_train_preprocessed, y_train, tc.BAYESIAN_OPTIMIZATION, base_pipeline, n_iter=10, cv=5)



Training and evaluating Logistic Regression with Bayesian Optimization:
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will 

/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


...

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [50, 1, 2] before, using random point [32, 3, 6]
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Bayesian Optimization Best Result for Decision Tree: 0.7854177622727837

Training and evaluating Random Forest with Bayesian Optimization:
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors 

/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [28, 1, 2, 199] before, using random point [26, 2, 7, 57]
  warnings.warn(


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
dirpath = "/Users/patricia/Documents/code/python/behavior-detection/output/"

reports = evaluation.generate_reports(trained_models, X_train_preprocessed, y_train, X_test_preprocessed, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)